In [167]:
import pandas as pd
import numpy as np
import os
from random import uniform,seed
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
import re
from sklearn.preprocessing import StandardScaler
from lifelines import CoxPHFitter
from bioinfokit.analys import stat
from random import randint
from scipy.spatial import distance

In [2]:
os.chdir('../')


In [3]:
from mc_hammer.mchammer import mchammer
from mc_hammer.null_distributions import pca_trans, random_order, min_max
from mc_hammer.clustering_algorithms import k_means
from mc_hammer.similarity_functions import huberts_gamma, norm_gamma, sillhouette_euclidean,sillhouette_cosine, CH, DB
from mc_hammer.similarity_functions import dunn,S_Dbw,SD_score,IGP,BWC,CVNN,dunn_min
from mc_hammer.hypothesis_test import hypothesis_test

## Functions

In [4]:
def mc_hammer_test(x,null_method,cluster_method,k = None,eps = None,min_samples = None):
    mch = mchammer()
    mch.get_null_distributions(x,null_method,repeats = 100)
    return mch.get_q_scores(cluster_method,k = k,eps=eps,min_samples=min_samples)

In [5]:
def get_true_list(test_list,true_k_ind):
    test_list2 = test_list[:true_k_ind] + test_list[true_k_ind + 1:]
    if min(test_list) == test_list[true_k_ind]:
        return True
    else:
        return False
def get_true_dict(res_list,true_k_ind):
    res_dict = {i:[j[i] for j in res_list] for i in res_list[0].keys()}
    res_dict = {k:get_true_list(v,true_k_ind) for k,v in res_dict.items()}
    return res_dict

In [6]:
def gouss_dist(seed_n):
    seed(seed_n)
    std = uniform(0.1,0.3)
    null_arr = np.array([np.random.normal(1,std,100) for i in range(3)])
    return null_arr.T

In [7]:
def big_mc_hammer(
    x_list,
    null_method,
    cluster_method,
    k_list = [2,4,5],
    eps_list = [0.5],
    min_samples_list = [4,5,7],
    spef = False,
    true_k_ind = None
):
    if cluster_method == 'DBSCAN':
        true_list=[mc_hammer_test(
            l,
            null_method,
            cluster_method,
            k=None,
            eps = j,
            min_samples=i
        ) for l in x_list for i in min_samples_list for j in eps_list]
    else:
        if spef:
            true_list=[[mc_hammer_test(
                j,
                null_method,
                cluster_method,
                k = i,
                eps = None,
                min_samples = None
            ) for i in k_list] for j in x_list] 
            
            true_list = [get_true_dict(i,true_k_ind) for i in true_list]
        else:
            true_list=[mc_hammer_test(
                j,
                null_method,
                cluster_method,
                k = i,
                eps = None,
                min_samples = None
            ) for j in x_list for i in k_list]
        true_dict = {i:[true_list[j][i] for j in range(len(true_list))] for i in true_list[0].keys()}
        if spef == False:    
            true_dict = {k:[True if i <0.05 else False for i in v] for k,v in true_dict.items()}
        true_dict_res = {k:v.count(True)/len(v)*100 for k,v in true_dict.items()}
    return true_dict_res

In [187]:
def test_blobs(k,noise,seperation,test_true,spef = False,pca = False):
    print(str(k) + str(noise) + str(seperation))
    x_list =[make_classification(
        n_samples = 100,
        n_features = 10,
        n_informative = int((10-10*noise)),
        n_redundant = int(10*noise),
        n_classes = k,
        n_clusters_per_class = 1,
        class_sep = seperation,
        random_state = i
    )[0] for i in range(50)]
    null_method_list = ['pca_trans','random_order','min_max']
    cluster_method_list = ['K_Means']
    if pca: 
        x_list = [PCA(n_components = 0.9).fit_transform(i) for i in x_list]
    if test_true:
        k_range = [k]
    else:
        k_range = [2,4,5]
    
    if spef: 
        true_k_ind = k_range.index(k)
    else:
        true_k_ind = None
    results_dict = {i +'_' + j:big_mc_hammer(
        x_list = x_list,
        null_method = i,
        cluster_method = j,
        k_list = k_range,
        spef = spef,
        true_k_ind = true_k_ind
    ) for i in null_method_list for j in cluster_method_list}
    
    results_df = pd.DataFrame(results_dict).T
    return(results_df)

In [188]:
def cvis(df,labels,q_methods = 'All'):
    
    if q_methods == 'All':
        q_methods = ['huberts_gamma', 'norm_gamma', 'sillhouette_euclidean','sillhouette_cosine', 'CH', 'DB',
                             'dunn','S_Dbw','SD_score','IGP','BWC','CVNN','dunn_min']
    q_dict = {}
    for i in q_methods:
        res = []
        if i in ['BWC','dunn','dunn_min']:
            for j in range(len(labels)):
                res_small = eval(i + '(df[' + str(j) + '],labels[' + str(j) + '][0],labels[' + str(j) + '][1])')
                res.append(res_small)
        else:
            for j in range(len(labels)):
                res_small = eval(i + '(df['+str(j)+'],labels['+str(j)+'][0])')
                res.append(res_small)
        q_dict[i] = res
    return q_dict



In [189]:
def get_dists(data,cluster_n,repeats):
    x = data
    pca_list = [pca_trans(x,i) for i in range(repeats)]
    random_list = [random_order(x, i) for i in range(repeats)]
    minmax_list = [min_max(x,i) for i in range(repeats)]
    null_dist_dict = {
        'pca_trans':pca_list,
        'random_shuffle':random_list,
        'min_max':minmax_list
                     }
    labels_dict = {k:[k_means(i,cluster_n) for i in v] for k,v in null_dist_dict.items()}
    full_dict = {k:cvis(null_dist_dict[k],v) for k,v in labels_dict.items()}
    full_dict = {k:{str(cluster_n) + '_'  +k+ '_' + k2:v2 for k2,v2 in v.items()} for k,v in full_dict.items()}
    full_dict3 = {}
    for i in full_dict.values():
        full_dict3.update(i)
    out_df = pd.DataFrame(full_dict3)
    return out_df

In [190]:
def clean_files(df):
    x = df.drop(columns = 'Y')
    x = StandardScaler().fit_transform(x)
    x2= PCA(n_components = 0.9).fit_transform(x)
    return x2

In [191]:
def real_data_test(x,cluster_n,repeats):
    lab,cen = k_means(x,cluster_n)
    q_methods = ['huberts_gamma', 'norm_gamma', 'sillhouette_euclidean','sillhouette_cosine', 'CH', 'DB',
                                'dunn','S_Dbw','SD_score','IGP','BWC','CVNN']
    q_dict = {}
    for i in q_methods:
        res = []
        if i in ['BWC','dunn']:
                res_small = eval(i + '(x,lab,cen)')
                res.append(res_small)
        else:
                res_small = eval(i + '(x,lab)')
                res.append(res_small)
        q_dict[i] = res

    null_dist = {'random_order':[random_order(x, i) for i in range(repeats)],
                 'min_max':[min_max(x,i) for i in range(repeats)]}

    labels_dict = {k:[k_means(i,cluster_n) for i in v] for k,v in null_dist.items()}
    full_dict = {k:cvis(null_dist[k],v,['sillhouette_euclidean','CH','DB','BWC']) for k,v in labels_dict.items()}
    full_dict = {k:{k2:v2 + q_dict[k2] for k2,v2 in v.items()} for k,v in full_dict.items() }
    hyp_dict = {k:{k+ '_' +k2:[hypothesis_test(v2,k2)] for k2,v2 in v.items()} for k,v in full_dict.items() }
    for i in hyp_dict.values():
        q_dict.update(i)
    
    return q_dict
        

In [192]:
def pick_best(method,q_list,k_list):
    if method in ['huberts_gamma', 'norm_gamma', 'sillhouette_euclidean','sillhouette_cosine', 'CH','dunn','IGP','BWC']:
        return(k_list[q_list.index(max(q_list))])
    else:
        return(k_list[q_list.index(min(q_list))])

In [193]:
def full_real_test(x,k_range,repeats):
    k_range_list = [real_data_test(x,i,repeats) for i in k_range]
    k_range_list = {i:[j[i] for j in k_range_list] for i in k_range_list[0].keys()}
    k_range_list = {k:[j for i in v for j in i] for k,v in k_range_list.items()}
    opt_k = {k:pick_best(k,v,k_range) for k,v in k_range_list.items()}
    found_clusters = {k:[True if i >= 0.05 else False for i in v] for k,v in k_range_list.items() if ('random_order' in k) or ('min_max' in k)}
    return opt_k, found_clusters

In [194]:
def size_effect(k,mult,dist):
    if dist == 'gauss':
        x_list = [gouss_dist(i) for i in range(100)]
    else:
        x_list = [np.random.rand(100,3) for i in range(100)]
    x_list = [i*mult for i in x_list]
    labels = [k_means(i,k) for i in x_list]
    cvi_dict = cvis(x_list,labels)
    cvi_df = pd.DataFrame(cvi_dict)
    cvi_df['k'] = k
    cvi_df['mult'] = mult
    cvi_df['dist'] = dist
    return cvi_df

In [195]:
def get_centers(x,labs):
    centers_list = []
    for k in range(max(labs)+1):
        one_clust = x[labs==k,:]
        center = one_clust.mean(axis = 0)
        centers_list.append(center)
    cent_array = np.array(centers_list)
    return cent_array

In [196]:
def true_label_test(x,labels,repeats):
    lab = np.array(labels.replace(labels.unique(),range(labels.nunique())))
    cen = get_centers(x,lab)
    cluster_n = labels.nunique()
    q = dunn_min(x,lab,cen)

    null_dist = {'random_order':[random_order(x, i) for i in range(repeats)],
                 'min_max':[min_max(x,i) for i in range(repeats)]}

    labels_dict = {k:[k_means(i,cluster_n) for i in v] for k,v in null_dist.items()}
    full_dict = {k:[dunn_min(null_dist[k][i],v[i][0],v[i][1]) for i in range(len(v))] for k,v in labels_dict.items()}
    hyp_dict = {k:hypothesis_test(v + [q],'dunn_min') for k,v in full_dict.items() }
    return hyp_dict
        

In [197]:
def true_label_test_full(x,labels,repeats):
    lab = np.array(labels.replace(labels.unique(),range(labels.nunique())))
    cen = get_centers(x,lab)
    cluster_n = labels.nunique()
    q_methods = ['huberts_gamma', 'norm_gamma', 'sillhouette_euclidean','sillhouette_cosine', 'CH', 'DB',
                               'dunn','S_Dbw','SD_score','IGP','BWC','CVNN']
    q_dict = {}
    for i in q_methods:
        res = []
        if i in ['BWC','dunn']:
                res_small = eval(i + '(x,lab,cen)')
                res.append(res_small)
        else:
                res_small = eval(i + '(x,lab)')
                res.append(res_small)
        q_dict[i] = res

    null_dist = {'random_order':[random_order(x, i) for i in range(repeats)],
                 'min_max':[min_max(x,i) for i in range(repeats)]}

    labels_dict = {k:[k_means(i,cluster_n) for i in v] for k,v in null_dist.items()}
    full_dict = {k:cvis(null_dist[k],v,['sillhouette_euclidean','CH','DB','BWC']) for k,v in labels_dict.items()}
    full_dict = {k:{k2:v2 + q_dict[k2] for k2,v2 in v.items()} for k,v in full_dict.items() }
    hyp_dict = {k:{k+ '_' +k2:[hypothesis_test(v2,k2)] for k2,v2 in v.items()} for k,v in full_dict.items() }
    hyp_dict = {k2:v2 for v in hyp_dict.values() for k2,v2 in v.items()}
    return hyp_dict

In [198]:
def mc_hammer_dunn_min(x,k = None,eps = None,min_samples = None):
    mch = mchammer()
    mch.get_null_distributions(x,'min_max',repeats = 100)
    return mch.get_q_scores('K_Means',q_methods = ['dunn_min'],k = k,eps=eps,min_samples=min_samples)

In [199]:
def dunn_min_positive(k,noise,seperation,repeats):
    x_list =[make_classification(
    n_samples = 100,
    n_features = 10,
    n_informative = int((10-10*noise)),
    n_redundant = int(10*noise),
    n_classes = k,
    n_clusters_per_class = 1,
    class_sep = seperation,
    random_state = i
    )[0] for i in range(repeats)]
    
    res_list = [mc_hammer_dunn_min(i,k) for i in x_list]
    res_list = [list(i.values())[0] for i in res_list]
    res_list = [1 if i < 0.05 else 0 for i in res_list]
    return sum(res_list)/len(res_list)

In [200]:
def load_data(file_name,sets):
    out_file_list = os.listdir('data/raw/clustEHR_data/'+file_name +'/data_' + str(sets))
    out = [i for i in out_file_list if 'output' in i][0]
    df = pd.read_csv('data/raw/clustEHR_data/'+file_name +'/data_' + str(sets) +'/'+out+ '/cluster_data.csv',index_col = 0)
    labels_df = pd.read_csv('data/raw/clustEHR_data/'+file_name +'/data_' + str(sets) + '/'+out+'/labels.csv',index_col = 0)
    outcomes = pd.read_csv('data/raw/clustEHR_data/'+file_name +'/data_' + str(sets) + '/'+out+'/outcomes.csv',index_col = 0)
    out_dict = {
        'data':df,
        'labels':labels_df,
        'outcomes':outcomes
    }
    return out_dict

In [201]:
def anova(df,labels,var):
    anova_dic = pd.DataFrame({'clusters':[str(i) for i in labels],'res':df[var]})
    res = stat()
    res.anova_stat(df = anova_dic, res_var = 'res',anova_model = 'res~C(clusters)')
    return res.anova_summary.loc['C(clusters)','PR(>F)']

In [221]:
def means_dif(df,labels,var):
    means_df = pd.DataFrame({'clusters':labels,'res':df[var]})
    means_df = means_df.groupby('clusters').mean()
    mean_list = means_df['res'].to_list()
    mean_dif_list =[abs(i-j) for idxi, i in enumerate(mean_list)for idxj, j in enumerate(mean_list) if idxi != idxj]
    return np.mean(mean_dif_list)

In [222]:
def get_clust_outcomes(df,labs,outcomes,var_list):
    x = df.drop(columns = 'PATIENT')
    x_scale = StandardScaler().fit_transform(x)
    x_pca = PCA(n_components=0.9).fit_transform(x_scale)
    k = labs['code'].nunique()
    mch = mchammer()
    mch.get_null_distributions(x_pca,'min_max',repeats = 100)
    q = mch.get_q_scores('K_Means',['dunn_min'],k = k)
    k_labs = mch.x_labels
    outs = outcomes.drop(columns = ['PATIENT','DEATH_AGE','YEARS_TO_DEATH'])
    anova_dict = {i:anova(outs,k_labs[0],i) for i in outs.columns}
    anova_count = [1 if i < 0.05 else 0 for i in anova_dict.values()]
    anova_count = sum(anova_count)/len(anova_count)
    start_dict = {
        'n':len(labs),
        'q':q,
        'anova_count':anova_count
    }
    anova_dict = {k + '_sig':v for k,v in anova_dict.items() if k in var_list}
    mean_dict = {i + '_meandif':means_dif(outs,k_labs[0],i) for i in var_list}
    return {**start_dict,**anova_dict,**mean_dict}

## Experiment 1: no clusters

In [ ]:
null_method_list = ['pca_trans']
cluster_method_list = ['K_Means']
x_list = [np.random.rand(100,3) for i in range(100)]
null_results_dict = {i +'_' + j:big_mc_hammer(
    x_list = x_list,
    null_method = i,
    cluster_method = j
) for i in null_method_list for j in cluster_method_list}

In [ ]:
null_results_dict

In [ ]:
null_df = pd.DataFrame(null_results_dict).T
null_df.to_csv('data/processed/fullex/null_results.csv')

In [ ]:
null_method_list = ['min_max','random_order']
cluster_method_list = ['K_Means']
x_list = [gouss_dist(i) for i in range(100)]
null_gous_results_dict = {i +'_' + j:big_mc_hammer(
    x_list = x_list,
    null_method = i,
    cluster_method = j
) for i in null_method_list for j in cluster_method_list}

In [ ]:
null_gous_results_dict

In [ ]:
null_gous_df = pd.DataFrame(null_gous_results_dict).T
null_gous_df.to_csv('data/processed/fullex/null_gouss_results.csv')

## Experiment 2: Finding Clusters

In [19]:
k_list = [2,4,5]
noise_list = [0.1,0.3,0.5]
sep_list = [0.5,1,3]

In [ ]:
blob_test = {'k-' + str(i) +'_noise-' +str(j) + '_sep-' + str(l):test_blobs(
    k = i,
    noise = j,
    seperation = l,
    test_true = True
) for i in k_list for j in noise_list for l in sep_list}

In [ ]:
blob_test

In [ ]:
if os.path.isdir('data/processed/fullex/k_means_pos_test') == False:
    os.mkdir('data/processed/fullex/k_means_pos_test')
for k,v in blob_test.items():
    file_name = re.sub('\.','',k)
    v.to_csv('data/processed/fullex/k_means_pos_test/' + file_name +'.csv')

## Experiment 3: Idenitify cluster number 

In [20]:
for i in k_list:
    for j in noise_list:
        blob_test_sens = {}
        blob_test_sens = {'k-' + str(i) +'_noise-' +str(j) + '_sep-' + str(l):test_blobs(
            k = i,
            noise = j,
            seperation = l,
            test_true = False,
            spef = True
        )for l in sep_list}
        if os.path.isdir('data/processed/fullex/k_means_sens_test') == False:
            os.mkdir('data/processed/fullex/k_means_sens_test')
        for k,v in blob_test_sens.items():
            file_name = re.sub('\.','',k)
            v.to_csv('data/processed/fullex/k_means_sens_test/' + file_name +'.csv')

20.10.5


C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:143: RuntimeWarning: divide by zero encountered in double_scalars
  return sep/comp
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:143: RuntimeWarning: divide by zero encountered in double_scalars
  return sep/comp
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:143: RuntimeWarning: divide by zero encountered in double_scalars
  return sep/comp


20.11


C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:143: RuntimeWarning: divide by zero encountered in double_scalars
  return sep/comp


20.13


...

50.51


C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:143: RuntimeWarning: divide by zero encountered in double_scalars
  return sep/comp
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:143: RuntimeWarning: divide by zero encountered in double_scalars
  return sep/comp
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:143: RuntimeWarning: divide by zero encountered in double_scalars
  return sep/comp
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:143: RuntimeWarning: divide by zero encountered in double_scalars
  return sep/comp


50.53


C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:143: RuntimeWarning: divide by zero encountered in double_scalars
  return sep/comp
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:134: RuntimeWarning: invalid value encountered in double_scalars
  sep_range = [np.mean(i)/max(i) for i in sep_range]
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:134: RuntimeWarning: invalid value encountered in double_scalars
  sep_range = [np.mean(i)/max(i) for i in sep_range]
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:134: RuntimeWarning: invalid value encountered in double_scalars
  sep_range = [np.mean(i)/max(i) for i in sep_range]
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:134: RuntimeWarning: invalid value encountered in double_scalars
  sep_range = [np.mean(i)/max(i) for i in sep_range]
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:134: RuntimeWarning: invalid v

C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:134: RuntimeWarning: invalid value encountered in double_scalars
  sep_range = [np.mean(i)/max(i) for i in sep_range]
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:134: RuntimeWarning: invalid value encountered in double_scalars
  sep_range = [np.mean(i)/max(i) for i in sep_range]
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:134: RuntimeWarning: invalid value encountered in double_scalars
  sep_range = [np.mean(i)/max(i) for i in sep_range]
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:134: RuntimeWarning: invalid value encountered in double_scalars
  sep_range = [np.mean(i)/max(i) for i in sep_range]
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:134: RuntimeWarning: invalid value encountered in double_scalars
  sep_range = [np.mean(i)/max(i) for i in sep_range]
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions

In [21]:
if os.path.isdir('data/processed/fullex/k_means_sens_test') == False:
    os.mkdir('data/processed/fullex/k_means_sens_test')
for k,v in blob_test_sens.items():
    file_name = re.sub('\.','',k)
    v.to_csv('data/processed/fullex/k_means_sens_test/' + file_name +'.csv')

## Experiment 4: PCA

In [ ]:
blob_test_pca = {'k-' + str(i) +'_noise-' +str(j) + '_sep-' + str(l):test_blobs(
    k = i,
    noise = j,
    seperation = l,
    test_true = False,
    spef = True,
    pca = True
) for i in k_list for j in noise_list for l in sep_list}

In [ ]:
if os.path.isdir('data/processed/fullex/k_means_pca_test') == False:
    os.mkdir('data/processed/fullex/k_means_pca_test')
for k,v in blob_test_sens.items():
    file_name = re.sub('\.','',k)
    v.to_csv('data/processed/fullex/k_means_pca_test/' + file_name +'.csv')

## Experiment 5a: Testing parameters 

In [ ]:
data = np.random.rand(100,3)
cluster_list = [2,3,4,5]
for i in cluster_list:
    test = get_dists(data,i,1000)
    if os.path.isdir('data/processed/fullex/k_means_dist_test') == False:
        os.mkdir('data/processed/fullex/k_means_dist_test')
    file_name = 'cluster_n-' + str(i)
    test.to_csv('data/processed/fullex/k_means_dist_test/' + file_name +'.csv')
    

## Experiemnt 5b: Testing size effect

In [28]:
mult = [1,3,5,10]
dist = ['gauss','uniform']
clust_n = [2,4,5]

In [29]:
size_dict = {'mult_' + str(m) + '_dist_' + d + '_k_' + str(k):size_effect(k,m,d) for k in clust_n for m in mult for d in dist}

In [30]:
for k,v in size_dict.items():
    v.to_csv('data/processed/fullex/size_test/' + k +'.csv')

## Experiment 6: Real Life Datasets 

In [31]:
csv_list = os.listdir('data/raw/real_data_sets')
csv_list = [i for i in csv_list if '.csv' in i]
df_list = [pd.read_csv('data/raw/real_data_sets/' + i) for i in csv_list]
clean_list = [clean_files(i) for i in df_list]


In [32]:
df_table = pd.DataFrame({
    'Name':[re.sub('.csv','',i) for i in csv_list],
    'Features':[i.shape[1] - 1 for i in df_list],
    'Observations':[len(i) for i in df_list],
    'Number of clusters':[len(i['Y'].unique()) for i in df_list]
})

In [33]:
df_table

,Name,Features,Observations,Number of clusters
0,breast_cancer,30,569,2
1,ecoli,7,336,8
2,glass,9,214,6
3,iris,4,150,3
4,wine,13,178,3
5,yeast,8,1484,10


In [35]:
found_cluster_list = []
opt_k_list = []
k_range = [i for i in range(2,13)]
for idx, i in enumerate(clean_list):
    opt_k, found_clusters = full_real_test(i,k_range,500)
    opt_k_list.append(opt_k)
    
    clust_n = df_table.loc[idx,'Number of clusters']
    k_ind = k_range.index(clust_n)
    found_cluster2 = {k:v[k_ind] for k,v in found_clusters.items()}
    found_cluster_list.append(found_cluster2)
f_clust_df = pd.DataFrame(found_cluster_list)
opt_df = pd.DataFrame(opt_k_list)
f_clust_df.to_csv('data/processed/fullex/k_means_real_test/found_clusters.csv')
opt_df.to_csv('data/processed/fullex/k_means_real_test/opt_clusters.csv')

In [204]:
out_list = []
for i in range(len(clean_list)):
    new_dict = true_label_test_full(clean_list[i],df_list[i]['Y'],500)
    out_list.append(new_dict)
out_df = pd.DataFrame(out_list)
out_df.to_csv('data/processed/fullex/k_means_real_test/true_clusters.csv')

In [ ]:
hyp_list = [true_label_test(clean_list[i],df_list[i]['Y'],500) for i in range(len(clean_list))]

In [ ]:
hyp_df = pd.DataFrame(hyp_list)

In [218]:
out_df = pd.DataFrame(out_list)
out_df.to_csv('data/processed/fullex/k_means_real_test/true_clusters.csv')

## Dunn Min Tests

In [205]:
x_list = [gouss_dist(i) for i in range(100)]

In [206]:
dunn_neg = [mc_hammer_dunn_min(i,k) for i in x_list for k in [2,4,5]]
dunn_neg = [list(i.values())[0] for i in dunn_neg]

In [211]:
dunn_df = pd.DataFrame({'ind':[i for i in range(len(dunn_neg))],'res':dunn_neg})
dunn_df.to_csv('data/processed/fullex/dunn_results/no_clusters.csv')

In [208]:
len([i in range(len(dunn_neg))])

1

In [210]:
dunn_neg

[0.6633663366336633,
 0.09900990099009899,
 0.02970297029702973,
 0.28712871287128716,
 0.2277227722772277,
 0.7425742574257426,
 0.49504950495049505,
 0.28712871287128716,
 0.24752475247524752,
 0.00990099009900991,
 0.05940594059405946,
 0.02970297029702973,
 0.8910891089108911,
 0.9405940594059405,
 0.08910891089108908,
 0.05940594059405946,
 0.9405940594059405,
 0.0,
 0.9207920792079208,
 0.7227722772277227,
 0.9702970297029703,
 0.36633663366336633,
 0.4257425742574258,
 0.3564356435643564,
 0.13861386138613863,
 0.9306930693069306,
 0.801980198019802,
 0.8712871287128713,
 0.5742574257425743,
 0.9702970297029703,
 0.19801980198019797,
 0.02970297029702973,
 0.0,
 0.29702970297029707,
 0.14851485148514854,
 0.6534653465346535,
 0.900990099009901,
 0.5247524752475248,
 0.9108910891089109,
 0.09900990099009899,
 0.7524752475247525,
 0.9207920792079208,
 0.7920792079207921,
 0.5445544554455446,
 0.6732673267326732,
 0.0,
 0.12871287128712872,
 0.37623762376237624,
 0.0,
 0.0495049504

In [212]:
k_list = [2,4,5]
noise_list = [0.1,0.3,0.5]
sep_list = [0.5,1,3]

In [213]:
dunn_pos = {
    'k_' + str(k) + '_noise_' + str(n) + '_sep_'+str(s):dunn_min_positive(k,n,s,50) 
    for k in k_list for n in noise_list for s in sep_list
}

In [214]:
dunn_pos_df = pd.DataFrame({'cat':list(dunn_pos.keys()),'results':list(dunn_pos.values())})
dunn_pos_df['k'] = dunn_pos_df['cat'].apply(lambda x: re.sub('k_','',x))
dunn_pos_df['k'] = dunn_pos_df['k'].apply(lambda x: re.sub('_.*','',x))
dunn_pos_df['noise'] = dunn_pos_df['cat'].apply(lambda x: re.sub('.*noise_','',x))
dunn_pos_df['noise'] = dunn_pos_df['noise'].apply(lambda x: re.sub('_.*','',x))
dunn_pos_df['sep'] = dunn_pos_df['cat'].apply(lambda x: re.sub('.*_sep_','',x))
dunn_pos_df = dunn_pos_df.drop(columns = 'cat')
dunn_pos_df.to_csv('data/processed/fullex/dunn_results/clusters.csv')

## Outcomes Testing

In [215]:
file_list = os.listdir('data/raw/clustEHR_data')
data_list = [load_data(i,j) for i in file_list for j in range(4)]

In [216]:
col_list = [i['outcomes'].columns.to_list() for i in data_list]
col_list = [j for i in col_list for j in i]
col_set = set(col_list)
col_count = {i:col_list.count(i) for i in col_set}
col_df = pd.DataFrame({'col':list(col_count.keys()),'count':list(col_count.values())})
all_cols = col_df['col'][col_df['count'] == col_df['count'].max()].to_list()
all_cols.remove('PATIENT')
all_cols.remove('DEATH_AGE')
all_cols.remove('YEARS_TO_DEATH')

In [223]:
res_list = [get_clust_outcomes(i['data'],labs = i['labels'],outcomes=i['outcomes'],var_list=all_cols) for i in data_list]

C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:143: RuntimeWarning: divide by zero encountered in double_scalars
  return sep/comp
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:143: RuntimeWarning: divide by zero encountered in double_scalars
  return sep/comp
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:143: RuntimeWarning: divide by zero encountered in double_scalars
  return sep/comp
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:143: RuntimeWarning: divide by zero encountered in double_scalars
  return sep/comp
C:\Users\nonie\Documents\mc-hammer\mc_hammer\similarity_functions.py:143: RuntimeWarning: divide by zero encountered in double_scalars
  return sep/comp


In [224]:
res_df = pd.DataFrame(res_list)
res_df.to_csv('data/processed/fullex/clustEHR_res/results.csv')

## Testing

In [118]:
df = data_list[0]['data']
labs = data_list[0]['labels']
outcomes = data_list[0]['outcomes']

In [121]:
x = df.drop(columns = 'PATIENT')
x_scale = StandardScaler().fit_transform(x)
x_pca = PCA(n_components=0.9).fit_transform(x_scale)
k = labs['code'].nunique()
mch = mchammer()
mch.get_null_distributions(x_pca,'min_max',repeats = 100)
q = mch.get_q_scores('K_Means',['dunn_min'],k = k)
k_labs = mch.x_labels

In [123]:
k = labs['code'].nunique()

mch = mchammer()
mch.get_null_distributions(x_pca,'min_max',repeats = 100)
q = mch.get_q_scores('K_Means',['dunn_min'],k = k)
k_labs = mch.x_labels

In [151]:
k_labs

(array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [154]:
anova_dic = pd.DataFrame({'clusters':[str(i) for i in k_labs[0]],'res':outcomes[ 'Systolic Blood Pressure_RATE']})
res = stat()
res.anova_stat(df = anova_dic, res_var = 'res',anova_model = 'res~C(clusters)')
res.anova_summary.loc['C(clusters)','PR(>F)']

In [156]:
res = stat()
res.anova_stat(df = anova_dic, res_var = 'res',anova_model = 'res~C(clusters)')
res.anova_summary.loc['C(clusters)','PR(>F)']

0.06648068155887271

In [183]:
all_cols

['Body Height_RATE',
 'YEARS_TO_DEATH',
 'Leukocytes [#/volume] in Blood by Automated count_RATE',
 'Hematocrit [Volume Fraction] of Blood by Automated count_RATE',
 'Pain severity - 0-10 verbal numeric rating [Score] - Reported_RATE',
 'Platelet distribution width [Entitic volume] in Blood by Automated count_RATE',
 'Diastolic Blood Pressure_RATE',
 'Body Mass Index_RATE',
 'Hemoglobin [Mass/volume] in Blood_RATE',
 'MCHC [Mass/volume] by Automated count_RATE',
 'Erythrocyte distribution width [Entitic volume] by Automated count_RATE',
 'MCV [Entitic volume] by Automated count_RATE',
 'DEATH_AGE',
 'Platelets [#/volume] in Blood by Automated count_RATE',
 'Body Weight_RATE',
 'Systolic Blood Pressure_RATE',
 'MCH [Entitic mass] by Automated count_RATE',
 'Erythrocytes [#/volume] in Blood by Automated count_RATE',
 'Platelet mean volume [Entitic volume] in Blood by Automated count_RATE']

In [161]:
test_labs = np.array([randint(0,6) for i in range(len(x))])

In [175]:
means_df = pd.DataFrame({'clusters':test_labs,'res':outcomes[ 'Systolic Blood Pressure_RATE']})
means_df = means_df.groupby('clusters').mean()
mean_list = means_df['res'].to_list()
mean_dif_list =[abs(i-j) for idxi, i in enumerate(mean_list)for idxj, j in enumerate(mean_list) if idxi != idxj]
round(np.mean(mean_dif_list),2)

[100.98259737585599,
 104.51062333304196,
 103.41044736691974,
 109.82286943371308,
 106.28629784541141,
 107.52789043417089,
 101.99495673990158]

In [180]:
mean_dif_list =[abs(i-j) for idxi, i in enumerate(mean_list)for idxj, j in enumerate(mean_list) if idxi != idxj]

In [181]:
round(np.mean(mean_dif_list),2)

3.85

In [103]:
import importlib
import sys

In [105]:
importlib.reload(sys.modules['mc_hammer.mchammer'])

<module 'mc_hammer.mchammer' from 'C:\\Users\\nonie\\Documents\\mc-hammer\\mc_hammer\\mchammer.py'>

In [106]:
from mc_hammer.mchammer import mchammer

In [113]:
x

,Unnamed: 0,asthma_medication,Body Height_FIRST,breast_medication,Body Mass Index_FIRST,DALY_FIRST,QOL_FIRST,Body Weight_FIRST,QALY_FIRST,ONSET_AGE,...,RACE_hispanic,sore_throat,RACE_black,sinusitis,RACE_asian,Never smoker,RACE_native,RACE_white,bronchitis,PATIENT
0,0,36.0,134.9,0.0,14.2,0.20,1.00,25.8,7.80,4.016427,...,0.0,1,0.0,1,0.0,1,0.0,1.0,1,608d361e-33c8-4173-bc45-ce6a2682f8b7
1,1,44.0,129.7,0.0,24.0,0.20,1.00,40.4,8.80,3.017112,...,0.0,1,0.0,1,0.0,1,0.0,1.0,1,7704c2a1-c075-4398-b027-51c52f9892f8
2,2,56.0,163.1,0.0,19.4,3.00,0.70,51.5,11.00,4.016427,...,0.0,1,0.0,1,0.0,1,0.0,1.0,1,8ee8f7a8-07fa-488f-8068-0d65b98c5c97
3,3,36.0,125.7,0.0,17.5,0.10,1.00,27.7,6.90,4.016427,...,0.0,0,0.0,0,0.0,1,0.0,1.0,0,03d63d20-5911-4669-aca7-e904eeefe3c7
4,4,60.0,155.6,0.0,20.5,2.30,0.70,49.5,9.70,2.017796,...,1.0,1,0.0,1,0.0,1,0.0,0.0,1,c4d85ba3-0734-4415-9eef-f417c394bfef
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,0.0,182.8,10.0,30.4,8.24,0.64,101.5,60.16,65.954825,...,1.0,1,0.0,1,0.0,1,0.0,0.0,1,2111590a-75c6-4080-b8f7-d5d85d80b190
396,396,0.0,164.0,0.0,27.7,10.30,0.64,74.5,63.30,45.968515,...,1.0,1,0.0,1,0.0,0,0.0,0.0,0,092c04c9-af85-44e4-8734-bcfb50e1cfd2
397,397,0.0,166.7,0.0,29.7,7.40,0.40,82.4,46.60,49.965777,...,0.0,1,0.0,1,0.0,1,0.0,1.0,1,0f8b6eef-fcfd-4680-9d23-3020ec87c289
398,398,0.0,156.0,3.0,30.2,7.70,0.80,73.5,40.30,48.966461,...,0.0,1,0.0,1,0.0,1,0.0,1.0,1,640217e0-7867-4d87-877a-f84fe43328c9
